# Evaluation
This notebook contains code to reproduce the results presented in the Energy Informatics Open Journal. 

## Testing

In [ ]:
import random
import math
import warnings
import math
import numpy as np
import pandas as pd
import pandapower
import inspect
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox
from pathlib import Path

from threats2power.communication.network import CommNetwork
from threats2power.cyber.analysis import Analyzer
from threats2power.attackers.random_attacker import RandomAttacker

def run_experiment(seed:int=0, spec:str="Default", grid:str|Path="create_cigre_network_mv", grid_kwargs:dict={},
                   criticality=None, save_name:str|None=None,
                   param_name:str="children_per_parent", param_values:list=[2, 3, 5, 8, 13, 21, 34, lambda network: network.n_devices],
                   n_attacks:int=1000, flatten:bool=False, auto_compromise_children:bool=False, **kwargs):
    np.random.seed(seed); random.seed(seed)
    print(f"Seed: {seed}")

    if param_name == "n_devices" and grid is not None:
        raise ValueError(f"Grid should be undefined for device count to be set.")
    grid_map = {name:creator for name, creator in inspect.getmembers(pandapower.networks, predicate=inspect.isfunction)}
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", category=FutureWarning)
        if type(grid) is str:
            grid = grid_map.get(grid)(**grid_kwargs) if grid is not None else None
        else:
            grid = pandapower.from_json(grid)
        criticality = criticality(grid, verbose=False)[0] if criticality is not None else criticality
        spec_path = Path.cwd() / "specifications" / f"{spec.capitalize()}_specifications.json"
        network = CommNetwork(n_devices=kwargs.get("n_devices", 20),
                              n_entrypoints=kwargs.get("n_entrypoints", 1), child_no_deviation=kwargs.get("child_no_deviation", 0),
                              children_per_parent=kwargs.get("children_per_parent", 3),
                              sibling_to_sibling_comm=kwargs.get("sibling_to_sibling_comm", None),
                              criticality=criticality,
                              network_specs=spec_path, grid=grid)
    print(f"Number of Components: {network.n_components}")

    # Total number of attacks: no_of_components * N_ATTACKS
    param_values = [val(network) if inspect.isfunction(val) else val for val in param_values]
    analyzer = Analyzer(network)
    exp_desc = '_'.join([w.capitalize() for w in param_name.split('_')])
    kwarg_desc = '-'.join(f"{k}_{v}" for k,v in kwargs.items())
    save_name = save_name if save_name is not None else f"{exp_desc}" + (f"--{kwarg_desc}" if kwarg_desc else "")
    archive_path = Path.cwd() / "data" / f"{save_name}.npz"
    print(f"Archive Path: '{archive_path}'")

    def run_monte(event):
        print("Running New Monte Carlo Simulation (Estimated Time to Completion: 40 minutes)")
        if len(param_values) > 1:
            compromised_array, effort_array, criticality_array = analyzer.monte_carlo_multi_analysis(seed=seed, 
                                                                                    n_attacks=n_attacks,
                                                                                    child_no_deviation=0,
                                                                                    grid=grid,
                                                                                    vary_entrypoints=kwargs.get("vary_entrypoints", True),
                                                                                    effort_only=kwargs.get("effort_only", False),
                                                                                    criticality=criticality,
                                                                                    auto_compromise_children=auto_compromise_children,
                                                                                    param_name=param_name, 
                                                                                    param_values=param_values)
        else:
            compromised_array, effort_array, criticality_array = analyzer.monte_carlo_analysis(
                n_attacks=n_attacks, attacker_variant=RandomAttacker, budget=kwargs.get("budget",52), device_only=False, 
                sibling_to_sibling_comm=kwargs.get("sibling_to_sibling_comm", None), vary_entrypoints=kwargs.get("vary_entrypoints", True),
                auto_compromise_children=auto_compromise_children,
            )
        np.savez(archive_path, compromise=compromised_array, effort=effort_array, criticality=criticality_array) # .flatten()
        analyzer.plot_monte(save_name=save_name, figsize=((14, 16) if not math.isclose(np.mean(criticality_array), 0) else (14,12)), flatten=flatten)

    def load_previous(event):
        print("Loading Previous Session")
        arrays = np.load(archive_path)
        compromised_array = arrays.get("compromise")
        effort_array = arrays.get("effort")
        criticality_array = arrays.get("criticality", np.zeros_like(compromised_array))
        analyzer.res_monte = {**{"compromised":compromised_array, "effort":effort_array, "criticality":criticality_array},
                              **({} if len(param_values) == 1 else {"param_name":param_name, "param_values":param_values})}
        analyzer.plot_monte(save_name=save_name, figsize=(14, 16) if "criticality" in arrays else (14,12), flatten=flatten)

    run_button = Button(description="Run Monte", button_style="info", style=dict(font_size="Large"), continuous_update=False)
    run_button.on_click(run_monte)
    load_button = Button(description="Load Previous", button_style="info", style=dict(font_size="Large"), continuous_update=False, disabled=False if archive_path.exists() else True)
    load_button.on_click(load_previous)
    box = HBox([run_button, load_button])
    display(box)

### Redundancy
Investigate the effect of redundancy (no. of children per parent) on the compromise and effort distribution of a communication network. 

In [ ]:
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="children_per_parent", param_values=[2, 3, 5, 8, 13, 21, 34, lambda network: network.n_devices],
               flatten=True, auto_compromise_children=False, save_name="RedundancyWChildrenAutoCompromised",
               n_attacks=1000)

### Budget
Investigate the effect of increasing the budget of attackers on the same communication network.
Effort Only means probability-of-success on defenses is ignored, if the effort threshold is met then that component is always compromised.


In [ ]:
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], # 250, 500, 1000, 2500, 5000, 10000
               children_per_parent = 3,
               flatten=True,
               effort_only=True, # Ignore success distributions
               n_attacks=1000)

### Susceptibility

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity

run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="Susceptibility52",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Sibling-to-Sibling Communication


In [ ]:
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="sibling_to_sibling_comm", param_values=[False, "adjacent", "all"],
               vary_entrypoints=True,
               save_name="siblings",
               flatten=True,
               n_attacks=10000)

### Vary Entrypoints
Show effect of attacking from different entrypoints, using colorscale instead of a Legend.

In [ ]:
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="VariedEntrypoints",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=False,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Static Analysis

#### Scalability

In [ ]:
import time
import seaborn as sns
from tqdm.notebook import trange
REPETITIONS = 100
device_counts = [5,4,3,2]
spec_path = Path.cwd() / "specifications" / "Default_specifications.json"
time_taken = {}
for i in trange(REPETITIONS):
    for j, n_devices in enumerate(device_counts):
        network = CommNetwork(n_devices=n_devices, n_entrypoints=1, children_per_parent=np.random.randint(1,n_devices), child_no_deviation=np.random.randint(1,n_devices), sibling_to_sibling_comm="all",
                            network_specs=spec_path)
        n_comp = network.n_components
        analyzer = Analyzer(network)
        # Time how long static analysis takes to complete
        start = time.perf_counter()
        _ = analyzer.static_analysis(verbose=False)
        end = time.perf_counter()
        time_taken[n_comp] = [end-start] if n_comp not in time_taken else time_taken[n_comp] + [end-start]
        # print(f"No. of Devices: {n_devices}, Components: {n_comp}, Time Taken: {end-start}s")

In [ ]:
fig = plt.figure(figsize=(6,5))
avg_time_taken = pd.DataFrame([{i:np.mean(vals) for i,vals in time_taken.items()}])

sns.scatterplot(x=[0,1,2,3,4,5,6], y=avg_time_taken.T.sort_index()[0], marker='X', color="k", zorder=4, legend=None, ax=plt.gca())
sns.swarmplot(time_taken, size=2, log_scale=True, ax=plt.gca())
plt.gca().set(xlabel="No. of Components in Network", ylabel="Time Taken to Analyze (s)")
plt.tight_layout()
plt.savefig(Path.cwd() / "media" / "Scalability.pdf")

#### Comparison to Monte Carlo Approach

In [ ]:
network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=2, child_no_deviation=0, sibling_to_sibling_comm="all",
                      network_specs=spec_path)
analyzer = Analyzer(network)
static_distr = analyzer.static_analysis(verbose=False, show_paths=False)
attack_counts = [1, 10, 100, 1000, 10000] # Per Entrypoint
monte_distrs = {}
for i, n_attacks in enumerate(attack_counts):
    comp_distr, *_ = analyzer.monte_carlo_analysis(n_attacks=n_attacks, budget=999999999, device_only=False, vary_entrypoints=True)
    vals, counts = np.unique(comp_distr, return_counts=True)
    monte_distr = {val:count/np.sum(counts) for val, count in zip(vals, counts)}
    monte_distrs[n_attacks] = monte_distr

In [ ]:
df = []
for key, distr in monte_distrs.items():
    for n_comp, prob in distr.items():
        df.append([f"Monte w. {key} runs" if isinstance(key, int) else f"Static Analysis", n_comp, prob])
df = pd.DataFrame(df, columns=["Type", "Components", "Probs"])
fig = plt.figure(figsize=(6,5))
sns.barplot(df, x="Components", y="Probs", hue="Type", errorbar=None, gap=0)
plt.gca().set(xlabel="No. of Components Compromised", ylabel="Probability")
plt.tight_layout()
plt.savefig(Path.cwd() / "media" / "StaticAnalysis.pdf")
plt.show()

### Use on Real Grid

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[52], save_name="SpanishLVSusceptibility",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True,
               criticality=criticality_by_capacity, 
               n_attacks=1000)

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)